In [3]:
import torch
#from denoising_diffusion_pytorch import Unet, GaussianDiffusion
#from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D
from denoising_diffusion_pytorch import *
import numpy as np
import matplotlib.pyplot as plt
#from scipy.stats import skew,kurtosis
from PIL import Image
import cv2

In [ ]:
pip install torch torchvision numpy matplotlib einops ema_pytorch accelerate pytorch_fid

## Load the data

In [5]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [6]:
data = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [7]:
# (40, 40, 99) (videonumber/trialnumber , channelnumber, datapoints)
data['sub01'].shape

(40, 40, 99)

In [5]:
# matrix 1
data['sub01'][0].shape

(40, 99)

In [8]:
matrix = np.zeros((1280,3,40,104))
matrix[:] = np.nan
counter = 0
check = np.zeros((104,40))
for sub in data.keys():
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        check = temp
        matrix[counter, :] = temp
        counter = counter + 1

In [9]:
matrix.shape

(1280, 3, 40, 104)

In [62]:
check[0]

array([0.34193571, 0.80623188, 0.06811771, 0.90654791, 0.09854338,
       0.04772348, 0.08286564, 0.15040094, 0.30077994, 0.65387255,
       0.69462309, 0.06582711, 0.90572575, 0.14277261, 0.08427975,
       0.08188165, 0.89104112, 0.01714881, 0.57187982, 0.41246195,
       0.04185219, 0.75087426, 0.09848531, 0.05350247, 0.13487976,
       0.36188846, 0.36149486, 0.53913732, 0.39823617, 0.03155432,
       0.98171454, 0.05919493, 0.03148297, 0.02311594, 0.71712978,
       0.02135819, 0.46509931, 0.5838429 , 0.08874375, 0.82108996,
       0.06263677, 0.03374082, 0.12594113, 0.32879547, 0.3429113 ,
       0.5032681 , 0.483895  , 0.02954557, 0.96045352, 0.04326873,
       0.01980131, 0.01610539, 0.87665463, 0.02988822, 0.36464224,
       0.38449448, 0.35547156, 0.77140733, 0.11058201, 0.06742717,
       0.26676349, 0.6188545 , 0.10892795, 0.66521184, 0.22899971,
       0.0921358 , 0.90980782, 0.04263698, 0.02005452, 0.06971488,
       0.37799671, 0.15096301, 0.51746719, 0.15454107, 0.05629

In [8]:
np.isnan(matrix).any()

False

In [64]:
matrix[0][0]

array([[0.29633097, 0.5152931 , 0.05009588, ..., 0.        , 0.        ,
        0.        ],
       [0.540126  , 0.55177657, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.55221434, 0.31579877, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.34482342, 0.46382535, 0.1260563 , ..., 0.        , 0.        ,
        0.        ],
       [0.90617533, 0.94400325, 0.67083414, ..., 0.        , 0.        ,
        0.        ],
       [0.35146873, 0.27318548, 0.20812183, ..., 0.        , 0.        ,
        0.        ]])

In [9]:
tensor_matrix = torch.tensor(matrix).float()

In [10]:
tensor_matrix.shape

torch.Size([1280, 3, 40, 104])

In [13]:
matrix.shape

(1280, 3, 40, 104)

In [71]:
!mkdir images
for i in range(1280):
    transpose = np.transpose(matrix[i,:,:,:], (1,2,0))
    tra = np.multiply(transpose, 255).astype(np.uint8)
    img = Image.fromarray(tra)
    img.save("images/train"+str(i)+".png")

### PNG and JPEG comparison

In [ ]:
test = cv2.imread('train20.jpeg')
print(test.shape)
test = np.transpose(test, (2,0,1))
test = np.multiply(test,1/255)
test

In [59]:
test2 = cv2.imread('train20.png')
print(test.shape)
test2 = np.transpose(test2, (2,0,1))
test2 = np.multiply(test2,1/255)
test2

(3, 40, 104)


array([[[0.29411765, 0.51372549, 0.04705882, ..., 0.        ,
         0.        , 0.        ],
        [0.5372549 , 0.54901961, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.54901961, 0.31372549, 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.34117647, 0.4627451 , 0.1254902 , ..., 0.        ,
         0.        , 0.        ],
        [0.90588235, 0.94117647, 0.67058824, ..., 0.        ,
         0.        , 0.        ],
        [0.34901961, 0.27058824, 0.20784314, ..., 0.        ,
         0.        , 0.        ]],

       [[0.29411765, 0.51372549, 0.04705882, ..., 0.        ,
         0.        , 0.        ],
        [0.5372549 , 0.54901961, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.54901961, 0.31372549, 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.34117647, 0.4627451 , 0.1254902 , ..., 0.        ,
         0.        , 0.        ],
        [0.9

In [69]:
np.allclose(fo, test2, rtol=0.1, atol=0.1)

True

In [ ]:
np.array_equal(fo,np.multiply(test,(1/255)))

For two images the png format was closer to the original input with a tolerance of 0.1 which failed in the case of jpeg. Therefore, png will be used to store the images

In [77]:
# 33 min for 50 train_num_steps
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8)
)

diffusion = GaussianDiffusion(
    model,
    image_size = (40,104),
    timesteps = 1000,   # number of steps
    loss_type = 'l1'    # L1 or L2
)

trainer = Trainer(
    diffusion,
    'images/',
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 50,#700000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
    calculate_fid = True              # whether to calculate fid during training
)

trainer.train()

  0%|          | 0/50 [00:00<?, ?it/s]

training complete


In [78]:
sampled_images = diffusion.sample(batch_size = 1)

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

In [79]:
sampled_images.shape

torch.Size([1, 3, 40, 104])

In [ ]:
# Training
# 880 images needs 880 / 3 layers = 294 
# 294 images needed into 10 chunks --> 30 images per loop

# Reconstruct Dataset
# 1280 images needs 1280 / 3 layers = 427
# 427 images needed into 10 chunks --> 43 images per loop

for i in range(10):
    sampled_images = diffusion.sample(batch_size = 43)
    torch.save(sampled_images,'sampledT_40-104_43_'+str(i))
    print("Finished Sample: ",i)

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

In [17]:
type(sampled_images)
torch.save(sampled_images,'test')

In [19]:
test = torch.load('test')
test.shape

torch.Size([427, 3, 40, 104])

In [21]:
test02 = torch.load("sampled_test0")
test02.shape

torch.Size([2, 3, 40, 104])